In [1]:
"""
GRASS GIS example
"""


obspoints = '/home/useriam/pwgis/geodata/mod2demo/obspoints.shp'
mdtraster = '/home/useriam/pwgis/geodata/mod2demo/mdt.tif'
outvis    = '/home/useriam/pwgis/geodata/mod2demo/visibility.tif'

In [2]:
import geopandas as gp
import os
from pycode.wenv import run_grass

In [6]:
# Create GRASS GIS Location
workspace = os.path.dirname(outvis)
location  = 'loc_' + os.path.splitext(os.path.basename(outvis))[0]

gbase = run_grass(workspace, location=location, srs=mdtraster)

# Start GRASS GIS Session

import grass.script.setup as gsetup
gsetup.init(gbase, workspace, location, 'PERMANENT')

'/tmp/tmp2ze2efkt'

In [7]:
from pycode.grstools import rst_to_grs
from pycode.grstools import grs_viewshed
from pycode.grstools import rstcalc
from pycode.grstools import grs_to_rst

In [8]:
# Add DEM to GRASS GIS workspace

grsdem = rst_to_grs(mdtraster, 'dem')

# Read Observation Points Shape
points = gp.read_file(obspoints)

In [9]:
points

,Id,geometry
0,0,POINT (309107.568 3623001.351)
1,0,POINT (312219.730 3622760.000)
2,0,POINT (310581.081 3620638.649)
3,0,POINT (311140.000 3623928.649)
4,0,POINT (313845.676 3623814.324)


In [10]:
visibilities = []
for idx, row in points.iterrows():
    # Produce viewshed raster
    vrst = grs_viewshed(
        grsdem, (row.geometry.x, row.geometry.y),
        f"visrst_{str(idx)}",
        max_dist=10000
    )
    visibilities.append(vrst)

In [11]:
print(visibilities)
print(" + ".join(visibilities))

['visrst_0', 'visrst_1', 'visrst_2', 'visrst_3', 'visrst_4']
visrst_0 + visrst_1 + visrst_2 + visrst_3 + visrst_4


In [13]:
name = visibilities[0]
for i in range(1, len(visibilities)):
    name = name + ' + ' + visibilities[i]
print(name)

visrst_0 + visrst_1 + visrst_2 + visrst_3 + visrst_4


In [14]:
# Sum all visibility rasters
fraster = rstcalc(" + ".join(visibilities), 'joinvis')

In [15]:
# Export result
grs_to_rst(fraster, outvis)

'/home/useriam/pwgis/geodata/mod2demo/visibility.tif'

In [16]:
# Export all temp files

for r in visibilities:
    grs_to_rst(r, os.path.join(workspace, r + '.tif'))